<a href="https://colab.research.google.com/github/charuj/annotated_transformer/blob/master/building_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf


In [ ]:
class LayerNorm:
  """ Applies layer normalization to sublayer. 
  Based on https://arxiv.org/abs/1607.06450
  
  set :
  epsilon= 1e-6 
  scope= "ln"
  """

  def __init__(self, inputs, epsilon, scope):
    super(LayerNorm, self).__init__()
    self.inputs= inputs
    self.eps= epsilon 
    self.scope= scope 

  def ln(self):
    ## CAN BE DONE IN KERAS INSTEAD 




def scaled_dot_product_attention(query, key, value, mask= None):
  """ Compute scaled dot product attention 
  query: shape (..., seqlen_q, depth_k)
  key: shape(..., seqlen_k, depth_k)
  value: shape(..., seqlen_v, depth_v)
  mask: float tensor, shape (..., seqlen_q, seqlen_k)

  """
  #Matrix multiplication of q & k
  qk= tf.matmul(q, k, transpose_b= True)
  #scale 
  d_k= tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_logits= qk/tf.sqrt(d_k)
  #add mask to the tensor 
  if mask is not None:
    scaled_logits += (mask * -1e-9) #mask is multiplied with scaler that is close to negative infinity 
  
  #softmax 
  #shape of logits: (..., seqlen_q, seqlen_k)
  attention_weights =tf.nn.softmax(scaled_logits, axis=-1) #normalize on last axis, seqlen_k
  outputs= tf.matmul(attention_weights, value) # becomes shape (..., seqlen_q, depth_v)

  return outputs, attention_weights






  

